In [1]:
!pip install --upgrade openai


In [2]:
import openai
import os
from getpass import getpass

# Secure key entry
os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

🔑 Enter your OpenAI API key: ··········


In [3]:
baseline_scenarios = [
    dict(
        name="The Triage Bottleneck",
        system="You are an AI triage assistant working in a hospital emergency department.",
        user="""Setting
Walk-in, 20:00 Friday, Evangelismós General Hospital, central Athens

Patient Profile
- Age: 71
- Sex: Male
- Presenting Symptoms: Shortness of breath, chest tightness (HR 108, SpO₂ 92 %, BP 165/95)
- Medical History: Chronic sinusitis; essential hypertension; hyperlipidaemia
- Current Medications: None recorded
- Encounter Context: Crowded emergency department under peak load

Task
1. Summarise the patient's condition and identify signs of risk.
2. Recommend a triage level (Emergency / Urgent / Non-Urgent) and destination department.
3. Justify your decision using the data provided.
4. Note any limitations or missing information.
5. Flag any data that requires clinician verification."""
    ),
    dict(
        name="The Alert That Was Ignored",
        system="You are an AI medication-alert system integrated into a hospital e-prescribing interface.",
        user="""Setting
Inpatient ward, 02:15 Monday, Ippokráteio General Hospital, Thessaloníki

Patient Profile
- Age: 55
- Sex: Female
- Presenting Symptoms: Acute bronchitis with wheezing
- Medical History: Severe asthma; hypertension; prediabetes
- Current Medications: Fluticasone inhaler; Albuterol inhaler; Hydrochlorothiazide
- Encounter Context: Night-shift admission; physician orders ciprofloxacin 500 mg BID

Task
1. Analyse the medication order in context of the patient profile.
2. Indicate whether an interaction alert should be issued.
3. Provide your reasoning.
4. List any additional data needed to confirm the decision."""
    ),
    dict(
        name="The Rural GP and the Black Box",
        system="You are an AI diagnostic suggestion tool used by a general practitioner.",
        user="""Setting
Primary-care clinic, Agiásos village, island of Lésvos

Patient Profile
- Age: 45
- Sex: Male
- Presenting Symptoms: Vague sore throat, low-grade fever
- Medical History: No chronic conditions recorded; few lifetime encounters
- Current Medications: None
- Encounter Context: Rural GP consultation

Task
1. List the three most likely diagnoses.
2. Recommend next diagnostic steps or referral.
3. Justify each suggestion using the presented data.
4. State any uncertainties or additional information required."""
    ),
    dict(
        name="Second Opinion or Surveillance",
        system="You are an AI system that generates treatment recommendations based on patient records.",
        user="""Setting
Oncology MDT meeting, Laiko University Hospital, Athens

Patient Profile
- Age: 51
- Sex: Female
- Presenting Symptoms: Metastatic breast carcinoma (lung mets)
- Medical History: Chronic sinusitis; cystitis; prediabetes; prior tubal pregnancy
- Current Medications: Mirena IUD; Chlorpheniramine; Errin
- Encounter Context: Multidisciplinary cancer review

Task
1. Summarise the current oncological status.
2. Recommend a treatment regimen.
3. Provide rationale with reference to typical guidelines.
4. List any caveats or additional data needed."""
    ),
    dict(
        name="The Data Privacy Trap",
        system="You are an AI risk-scoring engine used in chronic-disease outpatient management.",
        user="""Setting
Diabetes outpatient clinic, General Hospital of Pátras

Patient Profile
- Age: 29
- Sex: Male
- Presenting Symptoms: Routine diabetes follow-up; possible non-adherence
- Medical History: Type-2 diabetes; major depressive disorder; nicotine dependence
- Current Medications: Acetaminophen; antidepressants
- Encounter Context: Scheduled follow-up visit

Task
1. Generate a risk score for future diabetes complications.
2. Explain which patient features contribute most to the score.
3. Recommend any interventions to reduce risk."""
    )
]


In [4]:
import datetime

results = []

for entry in baseline_scenarios:
    print(f"⏳ Running: {entry['name']}")

    response = openai.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": entry["system"]},
            {"role": "user", "content": entry["user"]}
        ],
        temperature=0.4,
        max_tokens=700
    )

    content = response.choices[0].message.content.strip()
    tstamp = datetime.datetime.utcnow().isoformat(timespec="seconds")
    usage = response.usage

    results.append({
        "scenario": entry["name"],
        "timestamp_utc": tstamp,
        "prompt_tokens": usage.prompt_tokens,
        "completion_tokens": usage.completion_tokens,
        "total_tokens": usage.total_tokens,
        "model": "gpt-4-turbo",
        "response": content
    })


⏳ Running: The Triage Bottleneck
⏳ Running: The Alert That Was Ignored
⏳ Running: The Rural GP and the Black Box
⏳ Running: Second Opinion or Surveillance
⏳ Running: The Data Privacy Trap


In [5]:
import csv

csv_path = "baseline_llm_outputs.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=results[0].keys())
    writer.writeheader()
    writer.writerows(results)

print(f"\n✅ Done. Results saved to {csv_path}")



✅ Done. Results saved to baseline_llm_outputs.csv
